In [2]:
from selenium import webdriver  
import time  
import urllib  
from slugify import slugify
import requests
import random
import os

In [3]:

"""
some variables needed to be set up by users
"""
conference = 'siggraph'
conference_url = "https://dl.acm.org/toc/tog/2017/36/4" # the conference url to download papers from
chromedriver_path = 'C:\\Users\\xiaoyang\\Downloads\\chromedriver.exe' # the chromedriver.exe path
root = r'E:\OneDrive\academics\papers\conferences\SIGGRAPH-2017-ALL'.replace('\\','/') # file path to save the downloaded papers

os.makedirs(root, exist_ok=True)
print(root)

E:/OneDrive/academics/papers/conferences/SIGGRAPH-2017-ALL


In [4]:
"""
siggraph urls examples

siggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2020)
siggraph: https://dl.acm.org/toc/tog/2019/38/4 (SIGGRAPH 2019)
siggraph: https://dl.acm.org/toc/tog/2018/37/4 (SIGGRAPH 2018)
siggraph: https://dl.acm.org/toc/tog/2017/36/4 (SIGGRAPH 2017)

"""




'\nconference urls examples\n\ncvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)\neccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)\neccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)\niccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)\nicml: http://proceedings.mlr.press/v119/ (ICML 2020)\nneurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)\niclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)\nsiggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2020)\nsiggraph: https://dl.acm.org/toc/tog/2019/38/4 (SIGGRAPH 2019)\nsiggraph: https://dl.acm.org/toc/tog/2018/37/4 (SIGGRAPH 2018)\nsiggraph: https://dl.acm.org/toc/tog/2017/36/4 (SIGGRAPH 2017)\n\n'

In [13]:
session_1 = driver.find_elements_by_class_name('accordion-tabbed')[1].find_elements_by_class_name('toc__section')[0]

In [30]:
(session_1.find_element_by_xpath('a').text.split(':')[1].lstrip())

'Imaginative imaging'

In [21]:
session_1.find_elements_by_class_name('issue-item__content')[0].find_element_by_xpath('div/h5').text

'Session details: Imaginative imaging'

In [38]:
def retrieve_from_siggraph(driver):
    pdfurllist =  []
    pdfnamelist = []
    sessions = []
    import time    
    elementllist =  driver.find_elements_by_class_name('accordion-tabbed')[1].find_elements_by_class_name('toc__section')
    num_papers = 0
    for i, session in enumerate(elementllist):
        session.click()
        time.sleep(3)
        
        session_list = {}
        session_list['session'] = session.find_element_by_xpath('a').text.split(':')[1].lstrip()
        session_list['pdfnamelist'] = []
        session_list['pdfurllist'] = []
        print('\nSession: ', session_list['session'])

        for j, paper_element in enumerate(session.find_elements_by_class_name('issue-item__content')):
            try:
                paper_name = paper_element.find_element_by_xpath('div/h5').text
                pdf_url = paper_element.find_element_by_class_name('red').get_attribute('href')
                print('\t', paper_name)
                session_list['pdfnamelist'].append(paper_name)
                session_list['pdfurllist'].append(pdf_url)
                num_papers += 1
            except Exception as e:
                print(e)
            
        sessions.append(session_list)
    return sessions, num_papers

In [40]:
driver = webdriver.Chrome(chromedriver_path)  
driver.get(conference_url)

print('Retrieving pdf urls. This could take some time...')
sessions, num_papers = retrieve_from_siggraph(driver)



Retrieving pdf urls. This could take some time...

Session:  Imaginative imaging
Message: no such element: Unable to locate element: {"method":"css selector","selector":".red"}
  (Session info: chrome=91.0.4472.106)

	 CoLux: multi-object 3D micro-motion analysis using speckle imaging
	 4D imaging through spray-on optics
	 Rainbow particle imaging velocimetry for dense 3D fluid velocity imaging
	 Epipolar time-of-flight imaging

Session:  Mappings and deformations
Message: no such element: Unable to locate element: {"method":"css selector","selector":".red"}
  (Session info: chrome=91.0.4472.106)

	 Scalable Locally Injective Mappings
	 Geometric optimization via composite majorization
	 Variance-minimizing transport plans for inter-surface mapping
	 Regularized kelvinlets: sculpting brushes based on fundamental solutions of elasticity

Session:  Learning to move
Message: no such element: Unable to locate element: {"method":"css selector","selector":".red"}
  (Session info: chrome=91.0

In [41]:
# check the retrieved urls
print('The first pdf urls:\n')
print(sessions[0]['pdfurllist'])
print('\nThe last pdf urls:\n')
print(sessions[-1]['pdfurllist'])


The first pdf urls:

['https://dl.acm.org/doi/pdf/10.1145/3072959.3073607', 'https://dl.acm.org/doi/pdf/10.1145/3072959.3073589', 'https://dl.acm.org/doi/pdf/10.1145/3072959.3073662', 'https://dl.acm.org/doi/pdf/10.1145/3072959.3073686']

The last pdf urls:

['https://dl.acm.org/doi/pdf/10.1145/3072959.3073688', 'https://dl.acm.org/doi/pdf/10.1145/3072959.3054740', 'https://dl.acm.org/doi/pdf/10.1145/3072959.3073626', 'https://dl.acm.org/doi/pdf/10.1145/3072959.3073619']


In [42]:
# check the retrieved paper titles

print('The first pdf titles:\n')
print(sessions[0]['pdfnamelist'])
print('\nThe last pdf titles:\n')
print(sessions[-1]['pdfnamelist'])


The first pdf titles:

['CoLux: multi-object 3D micro-motion analysis using speckle imaging', '4D imaging through spray-on optics', 'Rainbow particle imaging velocimetry for dense 3D fluid velocity imaging', 'Epipolar time-of-flight imaging']

The last pdf titles:

['Interactive design space exploration and optimization for CAD models', 'Interactive Design and Stability Analysis of Decorative Joinery for Furniture', 'Lightweight structure design under force location uncertainty', 'Design and volume optimization of space structures']


In [43]:
print('The number of papers is ', num_papers)


The number of papers is  165


In [44]:
# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
i = 0
for session in sessions:
    pdfurllist = session['pdfurllist']
    pdfnamelist = session['pdfnamelist']
    session_dir = slugify(session['session'])
    os.mkdir(root+'/'+session_dir)
    for _, url in enumerate(pdfurllist):
        if url != None :      
            pdfname = slugify(pdfnamelist[i])
            if os.path.isfile(root+'/'+pdfname+".pdf"):
                print('existed', i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
            else:
                print(i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
                data = requests.get(pdfurllist[i], timeout=80, headers=headers).content
                
                with open(root+'/'+session_dir+'/'+pdfname+".pdf", 'wb')  as f:
                    f.write(data)  
                _ = time.sleep(random.uniform(4,5)) # for anti-anti-crawler purpose
            i += 1

Start downloading
0 	 CoLux: multi-object 3D micro-motion analysis using speckle imaging 	 https://dl.acm.org/doi/pdf/10.1145/3072959.3073607
1 	 4D imaging through spray-on optics 	 https://dl.acm.org/doi/pdf/10.1145/3072959.3073589
2 	 Rainbow particle imaging velocimetry for dense 3D fluid velocity imaging 	 https://dl.acm.org/doi/pdf/10.1145/3072959.3073662
3 	 Epipolar time-of-flight imaging 	 https://dl.acm.org/doi/pdf/10.1145/3072959.3073686
0 	 Scalable Locally Injective Mappings 	 https://dl.acm.org/doi/pdf/10.1145/3072959.2983621
1 	 Geometric optimization via composite majorization 	 https://dl.acm.org/doi/pdf/10.1145/3072959.3073618
2 	 Variance-minimizing transport plans for inter-surface mapping 	 https://dl.acm.org/doi/pdf/10.1145/3072959.3073671
3 	 Regularized kelvinlets: sculpting brushes based on fundamental solutions of elasticity 	 https://dl.acm.org/doi/pdf/10.1145/3072959.3073595
0 	 DeepLoco: dynamic locomotion skills using hierarchical deep reinforcement learni